# Egocentric Video Mapper

Before starting make sure you have an available GPU. If you are unsure about whether you have an available GPU or if you want to check which GPU you will be working with run the next cell.

In [1]:
!nvidia-smi -L

GPU 0: NVIDIA GeForce GTX 1080 Ti (UUID: GPU-72b7ddec-7074-1583-0076-9281c065ae9d)


Fill out the path to the directory of the uncompressed `Timeseries Data + Scene Video` download from Cloud.

In [ ]:
neon_timeseries_dir = "Path/To/NeonTimeSeriesFolder"

Fill out the path of the corresponding alternative egocentric view recording (please make sure that both videos have the same orientation).

In [ ]:
alternative_vid_path = "Path/To/AlternativeVideo.ext"

Define an output directory where the different output files will be saved.

In [ ]:
output_dir = "Path/To/OutputFolder"

## Select configurations

You can control how often, in seconds, the egocentric video mapper recalculates new image matches between the videos.


<div class="alert alert-block alert-info">Note: Higher time thresholds lead to a reduction in computation time, however it may decrease mapping accuracy.</div>

If you leave the value at 0 then for every gaze measurement new image matches will be calculated (slowest option but most accurate one).

In [3]:
refresh_time_threshold_sec = 0.5

Set the `render_video` variable to True if you want to render the mapped gaze into a copy of the alternative egocentric video (at its original frame rate).

Similarly, set the `render_video_comparison` variable to True if you want to render both egocentric videos (Neon Scene and the alternative egocentric camera) side by side showing their respective gaze measurements.

In [4]:
render_video = False
render_video_comparison = False

## Advanced configurations

Here you can choose which optic flow algorithm will be used for video synchronization, and which image matcher algorithm that will guide the gaze mapping. To check which ones are available in this repo, check `src/pupil_labs/egocentric_video_mapper/optic_flow.py` and `src/pupil_labs/egocentric_video_mapper/feature_matcher.py`, respectively.

In [5]:
optic_flow_algorithm = "Lucas-Kanade"
image_matcher = "Efficient_LOFTR"

## Now let's run the egocentric video mapper!

When everything is finished, you will find the following in the specified output folder:
- `alternative_camera_gaze.csv`: The mapped gaze signal. It follows the same structure as gaze.csv, with the frequency of the gaze signal (200Hz) being preserved. This way you can easily integrate it into your existing pipelines.
- `alternative_camera_timestamps.csv`: Synchronized UTC timestamps for every alternative egocentric video frame. It follows the same structure as world_timestamps.csv
- `alternative_camera-neon_comparison.mp4`: The comparison video showing side by side the Neon Scene camera video and the alternative camera video with their respective gaze signal overlaid.
- `alternative_camera_gaze_overlay.mp4`: A copy of the alternative egocentric video with the mapped gaze overlaid.
- `neon_optic_flow.csv`: Contains the average optic flow csv for Neon Scene video.
- `alternative_optic_flow.csv`: Contains the average optic flow csv for alternative egocentric video.
- `egocentric_video_mapper_args.json`: This file contains the different parameters and configurations used to map the gaze signal

In [6]:
import pupil_labs.egocentric_video_mapper.__main__ as main
from pathlib import Path


class Args:
    def __init__(self):
        self.neon_timeseries_dir = neon_timeseries_dir
        self.alternative_vid_path = alternative_vid_path
        self.output_dir = output_dir
        self.optic_flow_choice = optic_flow_algorithm
        self.matcher = image_matcher
        self.refresh_time_thrshld = refresh_time_threshold_sec
        self.render_comparison_video = render_video_comparison
        self.render_video = render_video


args = Args()

main.main(args)

# Hi, so the Egocentric Video Mapper Alpha Lab is ready for internal testing, I created a Google Colab for use the tool

Calculating optic flow: 100%|██████████| 1291/1291 [01:04<00:00, 20.09it/s]


Estimated offset of alternative egocentric video with respect to Neon scene video: 2.016 seconds (Pearson correlation: 0.9192123717590794)


Mapping gaze signal: 100%|██████████| 8155/8155 [01:44<00:00, 77.88it/s] 

Gaze mapped to alternative camera video saved at /users/sof/test_main/test_note/alternative_camera_gaze.csv
